In [1]:
import pandas as pd
import numpy as np
import cv2
import dlib
import math
from time import time

In [2]:
df=pd.read_csv("../facial_expressions/data/legend.csv").drop("user.id",axis=1)

In [3]:
IMG_PATH="../facial_expressions/images/"

In [4]:
emotions=["neutral", "anger", "disgust", "fear", "happy", "sad"]

In [5]:
df["emotion"]=df["emotion"].map(lambda x: x.lower())

In [6]:
df=df[df.emotion!="surprise"]
df=df[df.emotion!="contempt"]

In [36]:
df[df.emotion!="sad"]

,image,emotion,index
0,facial-expressions_2868588k.jpg,anger,1
2,facial-expressions_2868584k.jpg,disgust,2
3,facial-expressions_2868582k.jpg,fear,3
4,Aaron_Eckhart_0001.jpg,neutral,0
5,Aaron_Guiel_0001.jpg,happy,4
...,...,...,...
14185,Tim_Pawlenty_0001.jpg,neutral,0
14186,Tim_Robbins_0001.jpg,neutral,0
14187,Tim_Robbins_0002.jpg,neutral,0
14188,Tim_Robbins_0003.jpg,neutral,0


In [7]:
df=df.replace(to_replace ="contempt", value ="disgust")
df=df.replace(to_replace ="surprise", value ="happy")
df=df.replace(to_replace ="happiness", value ="happy")
df=df.replace(to_replace ="sadness", value ="sad")

In [8]:
inds=[]
for em in df["emotion"]:
    inds.append(emotions.index(em))
df["index"]=inds

In [9]:
df

,image,emotion,index
0,facial-expressions_2868588k.jpg,anger,1
2,facial-expressions_2868584k.jpg,disgust,2
3,facial-expressions_2868582k.jpg,fear,3
4,Aaron_Eckhart_0001.jpg,neutral,0
5,Aaron_Guiel_0001.jpg,happy,4
...,...,...,...
14185,Tim_Pawlenty_0001.jpg,neutral,0
14186,Tim_Robbins_0001.jpg,neutral,0
14187,Tim_Robbins_0002.jpg,neutral,0
14188,Tim_Robbins_0003.jpg,neutral,0


In [10]:
face_cascade = cv2.CascadeClassifier('../face_recognition/haarcascade_frontalface_default.xml')
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [12]:
model=Sequential()
model.add(Dense(512,activation='relu', input_shape=(68*2,)))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(emotions),activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               70144     
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3

In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
face_dim=128

In [17]:
def ret_keypoints(imname):
    try:
        image = cv2.imread(IMG_PATH+imname)
    except:
        return None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (face_dim, face_dim))
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
#     boxes = [(y,x+w,y+h,x) for (x,y,w,h) in faces]
    shape=predictor(gray,dlib.rectangle(0,0,face_dim,face_dim))
    xlist=[]
    ylist=[]
    for i in range(68):
        xlist.append(float(shape.part(i).x))
        ylist.append(float(shape.part(i).y))
    xmean = np.mean(xlist)
    ymean = np.mean(ylist)
#     plt.imshow(gray,cmap='gray')
#     plt.scatter(xlist,ylist, marker='.')
#     plt.show()
    xcentral = [(x-xmean) for x in xlist]
    ycentral = [(y-ymean) for y in ylist]
    res=[]
    for (x,y) in zip(xcentral,ycentral):
        res.append(x)
        res.append(y)
    return np.asarray(res)

In [18]:
def prep_keypoints():
    X_inp=[]
    y_inp=[]
    for idx in range(len(df)):
        print("\r",idx+1,end=" ")
        row=df.iloc[idx]
        landmarks=ret_keypoints(row["image"])
        if landmarks is not None:
            X_inp.append(landmarks)
            yy=np.zeros(len(emotions))
            yy[row["index"]]=1
            y_inp.append(yy)
    return np.asarray(X_inp),np.asarray(y_inp)

In [19]:
XXD,YYD=prep_keypoints()

 13813 8414 

In [26]:
s=np.arange(len(XXD))
np.random.shuffle(s)
XXD=XXD[s]
YYD=YYD[s]

In [27]:
cut=800
XDB=XXD[cut:]
YDB=YYD[cut:]

XTB=XXD[:cut]
YTB=YYD[:cut]

In [28]:
model.fit(XDB,YDB,batch_size=32,epochs=10,validation_data=(XTB,YTB))

Train on 13013 samples, validate on 800 samples
Epoch 1/50
13013/13013 [==============================] - 2s 159us/sample - loss: 0.5538 - accuracy: 0.8138 - val_loss: 0.5815 - val_accuracy: 0.7912
Epoch 2/50
13013/13013 [==============================] - 2s 159us/sample - loss: 0.5574 - accuracy: 0.8136 - val_loss: 0.5215 - val_accuracy: 0.8313
Epoch 3/50
13013/13013 [==============================] - 2s 163us/sample - loss: 0.5589 - accuracy: 0.8086 - val_loss: 0.4761 - val_accuracy: 0.8425
Epoch 4/50
13013/13013 [==============================] - 2s 160us/sample - loss: 0.5587 - accuracy: 0.8130 - val_loss: 0.5063 - val_accuracy: 0.8400
Epoch 5/50
13013/13013 [==============================] - 2s 168us/sample - loss: 0.5525 - accuracy: 0.8137 - val_loss: 0.4918 - val_accuracy: 0.8238
Epoch 6/50
13013/13013 [==============================] - 2s 162us/sample - loss: 0.5581 - accuracy: 0.8154 - val_loss: 0.5494 - val_accuracy: 0.8087
Epoch 7/50
13013/13013 [============================

KeyboardInterrupt: 

In [29]:
(model.predict(XTB).argmax(axis=1)==YTB.argmax(axis=1)).sum()/len(XTB)

0.84625

In [30]:
model.predict(XTB).argmax(axis=1)

array([0, 4, 0, 4, 0, 4, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0,
       0, 0, 4, 4, 0, 0, 0, 4, 4, 0, 0, 0, 4, 4, 0, 4, 0, 4, 4, 0, 0, 0,
       0, 4, 4, 4, 4, 4, 0, 0, 4, 4, 1, 4, 4, 0, 0, 0, 0, 0, 4, 4, 4, 0,
       0, 4, 0, 0, 0, 0, 4, 0, 4, 0, 4, 4, 0, 4, 4, 0, 4, 4, 0, 0, 0, 4,
       4, 4, 4, 0, 4, 0, 0, 0, 4, 4, 0, 0, 4, 0, 4, 4, 4, 0, 4, 0, 4, 0,
       4, 4, 0, 0, 0, 4, 4, 0, 4, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4, 0,
       0, 4, 0, 4, 4, 4, 4, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4, 0, 0, 4, 4,
       4, 4, 4, 0, 4, 0, 4, 4, 0, 4, 4, 4, 0, 4, 4, 0, 4, 4, 4, 0, 0, 4,
       0, 4, 0, 4, 4, 0, 4, 0, 4, 4, 4, 4, 0, 4, 4, 4, 0, 0, 4, 0, 4, 0,
       0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 4, 0, 4, 4, 0, 0, 0, 0, 4,
       4, 4, 4, 4, 0, 0, 4, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0,
       0, 0, 4, 4, 0, 0, 0, 4, 0, 0, 0, 4, 4, 0, 0, 0, 4, 4, 4, 4, 0, 0,
       4, 0, 0, 0, 4, 0, 0, 4, 0, 0, 4, 4, 0, 4, 4, 1, 0, 0, 0, 0, 4, 4,
       4, 0, 4, 4, 1, 0, 4, 0, 4, 0, 4, 4, 0, 4, 0,

In [31]:
YTB.argmax(axis=1)

array([0, 4, 0, 4, 0, 4, 4, 0, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 4, 4, 4, 0, 4, 0, 4, 4, 0, 0, 0,
       0, 4, 5, 4, 4, 0, 0, 0, 4, 4, 1, 4, 4, 4, 0, 0, 0, 0, 4, 4, 4, 4,
       0, 4, 4, 4, 5, 0, 4, 0, 4, 0, 0, 4, 0, 4, 4, 0, 4, 4, 0, 4, 0, 0,
       4, 4, 4, 0, 4, 0, 4, 0, 4, 4, 0, 0, 4, 4, 4, 4, 4, 2, 4, 0, 4, 0,
       4, 4, 0, 0, 4, 0, 0, 4, 4, 0, 0, 2, 0, 4, 0, 4, 0, 4, 0, 0, 0, 0,
       0, 4, 0, 4, 4, 4, 4, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0,
       4, 4, 4, 0, 4, 0, 4, 4, 1, 2, 4, 4, 0, 4, 4, 0, 4, 4, 5, 0, 4, 4,
       0, 4, 0, 4, 4, 0, 4, 0, 4, 4, 4, 4, 0, 4, 4, 4, 0, 0, 4, 0, 4, 0,
       0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 4, 4, 0, 4, 0, 4, 4, 0, 0, 4, 5, 4,
       4, 4, 4, 4, 0, 0, 4, 4, 5, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       4, 1, 0, 4, 0, 0, 0, 4, 4, 3, 0, 4, 4, 0, 0, 0, 4, 0, 4, 4, 0, 0,
       0, 0, 0, 3, 4, 0, 4, 4, 0, 0, 0, 4, 0, 4, 4, 1, 0, 4, 0, 0, 4, 4,
       4, 0, 4, 4, 1, 0, 4, 0, 4, 0, 0, 4, 0, 4, 0,